In [135]:
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from keras.models import Model
from keras.layers import Input, Reshape, TimeDistributed
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling1D, Conv1D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
from keras.callbacks import TensorBoard
from music21 import *
import glob
import numpy as np
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.callbacks import History, ModelCheckpoint
from keras.optimizers import RMSprop
from datetime import datetime

In [136]:
path = '/home/eurismar/notebooks/Music_Generator_Demo/Pop_Music_Midi/'

In [137]:
files = glob.glob('{}/*.mid*'.format(path))

In [138]:
KEY_SIG_OFFSET = 0 
melody_corpus = []
for i, file in enumerate(files[:2]):
#for file in files:
    midi_stream = converter.parse(file)
    
    part0 = midi_stream[0]
    key_sig = part0.keySignature
    midi_stream.transpose(KEY_SIG_OFFSET - key_sig.tonic.pitchClass, inPlace=True)
    for m in part0.measures(1, None):
        if hasattr(m, 'flat'):
            for nr in m.flat.notesAndRests:
                pitch = nr.pitch.midi  if isinstance(nr, note.Note) else 0
                duration = float(nr.quarterLength)
                note_repr = (pitch, duration)
                melody_corpus.append(note_repr)

In [139]:
melody_set = set(melody_corpus)

In [140]:
notes_indices = {note: i for i, note in enumerate(melody_set)}
indices_notes = {i: note for i, note in enumerate(melody_set)}

In [141]:
corpus_size = len(melody_set)
print(corpus_size)

25


In [142]:
phrase_len = 20
step_size = 3

In [143]:
phrases = []
next_notes = []

In [144]:
for i in range(0, len(melody_corpus) - phrase_len, step_size):
    phrases.append(melody_corpus[i: i + phrase_len])
    next_notes.append(melody_corpus[i + phrase_len])
print('nb sequences:', len(phrases))

nb sequences: 43


In [145]:
# transform data into binary matrices
X = np.zeros((len(phrases), phrase_len, corpus_size), dtype=np.bool)
y = np.zeros((len(phrases), corpus_size), dtype=np.bool)
print(X.shape)
print(y.shape)

(43, 20, 25)
(43, 25)


In [146]:
for i, phrase in enumerate(phrases):
    for j, note in enumerate(phrase):
        X[i, j, notes_indices[note]] = 1
    y[i, notes_indices[next_notes[i]]] = 1

In [147]:
layer_size=128
batch_size = 128
nb_epoch = 400

In [148]:
corpus_size

25

In [149]:
phrase_len

20

In [154]:
def get_gen(G_in, phrase_len=phrase_len, corpus_size=corpus_size):
    #G_in = Input(shape=(phrase_len, corpus_size))
    layer = LSTM(layer_size, return_sequences=True)(G_in)
    layer = Dropout(0.2)(layer)
    layer = LSTM(layer_size, return_sequences=True)(layer)
    layer = Dropout(0.2)(layer)
    layer = TimeDistributed(Dense(corpus_size))(layer)
    G_out = Activation('softmax')(layer)
    LSTM_out = Activation('softmax')(layer)
    G = Model(G_in, G_out)
    G.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return G, G_out
G_in = Input(shape=(phrase_len, corpus_size))
G, G_out = get_gen(G_in)
G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20, 25)            0         
_________________________________________________________________
lstm_19 (LSTM)               (None, 20, 128)           78848     
_________________________________________________________________
dropout_18 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 20, 128)           131584    
_________________________________________________________________
dropout_19 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 25)            3225      
_________________________________________________________________
activation_10 (Activation)   (None, 20, 25)            0         
Total para

In [155]:
def get_dis(D_in, phrase_len=phrase_len, corpus_size=corpus_size):
    
    #G_in = Input(shape=(phrase_len, corpus_size))
    layer = LSTM(layer_size, return_sequences=True)(D_in)
    layer = Dropout(0.2)(layer)
    layer = LSTM(layer_size)(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(2)(layer)
    D_out = Activation('sigmoid')(layer)
    D = Model(D_in, D_out)
    D.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
    return D, D_out    

In [156]:
D_in = Input(shape=(phrase_len, corpus_size))
#G_in = Input(shape=(phrase_len, corpus_size))

In [157]:
D, D_out = get_dis(D_in)
D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 20, 25)            0         
_________________________________________________________________
lstm_21 (LSTM)               (None, 20, 128)           78848     
_________________________________________________________________
dropout_20 (Dropout)         (None, 20, 128)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_21 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 2)                 258       
_________________________________________________________________
activation_12 (Activation)   (None, 2)                 0         
Total para

In [158]:
def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable

In [159]:
def make_gan(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in) # ENTRADA DA GAN
    GAN_out = D(x)
    GAN = Model(GAN_in, GAN_out)
    GAN.compile(loss='binary_crossentropy', optimizer=G.optimizer)
    return GAN, GAN_out

In [160]:
GAN_in = Input(shape=(phrase_len, corpus_size))
GAN, GAN_out = make_gan(GAN_in, G, D)
GAN.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 20, 25)            0         
_________________________________________________________________
model_6 (Model)              (None, 20, 25)            213657    
_________________________________________________________________
model_7 (Model)              (None, 2)                 210690    
Total params: 424,347
Trainable params: 213,657
Non-trainable params: 210,690
_________________________________________________________________


In [161]:
n_samples = 10
noise_dim = 25

In [162]:
X.shape

(43, 20, 25)

In [163]:
y.shape

(43, 25)

In [176]:
def sample_data_and_gen(G,n_samples=n_samples,noise_dim=noise_dim):
    XT = X[:n_samples]
    XN_noise = np.random.uniform(0, 1, size=[n_samples,phrase_len,noise_dim])
    XN = G.predict(XN_noise)
    print(XT.shape,XN.shape)
    XX = np.concatenate((XT, XN))
    yy = np.zeros((2*n_samples, 2))
    yy[:n_samples, 1] = 1
    yy[n_samples:, 0] = 1
    return XX, yy

In [177]:
def pretrain(G, D, noise_dim=10, n_samples=10000, batch_size=32):
    X, y = sample_data_and_gen(G, n_samples=n_samples, noise_dim=noise_dim)
    set_trainability(D, True)
    D.fit(X, y, epochs=1, batch_size=batch_size)

In [178]:
pretrain(G, D,noise_dim = noise_dim, n_samples=n_samples,batch_size=1)

(10, 20, 25) (10, 20, 25)
Epoch 1/1
20/20 [==============================] - 0s - loss: 0.0024 - acc: 1.0000     


In [189]:
def sample_noise(G, noise_dim=10, n_samples=10000):
    X = np.random.uniform(0, 1, size=[n_samples,phrase_len, noise_dim])
    y = np.zeros((n_samples, 2))
    y[:, 1] = 1
    return X, y

In [190]:
d_loss = []
g_loss = []

In [191]:
d_loss = []
for epoch in range(2):
    X, y = sample_data_and_gen(G, n_samples=10, noise_dim=noise_dim)
    set_trainability(D, True)
    d_loss.append(D.train_on_batch(X, y))
    X, y = sample_noise(G, n_samples=n_samples, noise_dim=noise_dim)
    set_trainability(D, False)

(10, 25) (10, 20, 25)


ValueError: all the input arrays must have same number of dimensions

In [106]:
def train(GAN, G, D, epochs=500, n_samples=10000, noise_dim=10, batch_size=32, verbose=False, v_freq=50):
    d_loss = []
    g_loss = []
    e_range = range(epochs)
    if verbose:
        e_range = tqdm(e_range)
    for epoch in e_range:
        #XT = X[:n_samples]
        X, y = sample_data_and_gen(G, n_samples=10, noise_dim=noise_dim)
        set_trainability(D, True)
        d_loss.append(D.train_on_batch(X, y)) #TREINAMENTO ***************************************************
        
        X, y = sample_noise(G, n_samples=n_samples, noise_dim=noise_dim)
        set_trainability(D, False)
        g_loss.append(GAN.train_on_batch(X, y))
        if verbose and (epoch + 1) % v_freq == 0:
            print("Epoch #{}: Generative Loss: {}, Discriminative Loss: {}".format(epoch + 1, g_loss[-1], d_loss[-1]))
    return d_loss, g_loss

In [107]:
d_loss, g_loss = train(GAN, G, D, verbose=True)

A Jupyter Widget

ValueError: Error when checking : expected input_1 to have shape (None, 20, 25) but got array with shape (10, 20, 10)

In [ ]:
ax = pd.DataFrame(
    {
        'Generative Loss': g_loss,
        'Discriminative Loss': d_loss,
    }
).plot(title='Training loss', logy=True)
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")

In [ ]:
N_VIEWED_SAMPLES = 2
data_and_gen, _ = sample_data_and_gen(G, n_samples=N_VIEWED_SAMPLES)
pd.DataFrame(np.transpose(data_and_gen[N_VIEWED_SAMPLES:])).plot()